In [7]:
import pandas as pd
import numpy as np
from math import e
from numpy import log as ln
from datetime import date

In [8]:
df = pd.read_csv("2018to2019_comedy(23).csv")

In [9]:
df.head()

,Unnamed: 0.1,Unnamed: 0,channel_title,video_title,published_time,category,tags,description,view,likes,comments,duration,N_cal,published_time_view
0,0,1,The Tonight Show Starring Jimmy Fallon,Tones and I: Dance Monkey (US TV Debut),2019-11-19T12:00:02Z,23,"['The Tonight Show', 'Jimmy Fallon', 'Tones an...",Music guest Tones and I makes her US TV debut ...,109541509,2121432,83661,PT4M8S,1.095426e+08,1.021769e+08
1,1,2,The Tonight Show Starring Jimmy Fallon,Musical Genre Challenge with Ariana Grande,2018-05-02T04:15:27Z,23,"['The Tonight Show', 'Jimmy Fallon', 'Musical ...","Ariana Grande transforms Drake's ""God's Plan"" ...",91136025,1677410,52531,PT5M29S,9.113694e+07,8.500885e+07
2,2,3,The Tonight Show Starring Jimmy Fallon,Jack Black Performs His Legendary Sax-A-Boom w...,2018-09-21T10:00:06Z,23,"['The Tonight Show', 'Jimmy Fallon', 'TSJF Int...",Jack Black gives The Tonight Show audience a t...,88961102,2149817,38017,PT2M22S,8.896199e+07,8.298015e+07
3,3,4,The Tonight Show Starring Jimmy Fallon,"BTS: ""Idol"" | The Tonight Show Starring Jimmy ...",2018-09-26T04:30:41Z,23,"['The Tonight Show', 'Jimmy Fallon', 'BTS', 'P...",Global phenomenon BTS (방탄소년단) delivers a high-...,47601044,1243350,44123,PT4M9S,4.760152e+07,4.440077e+07
4,4,5,The Tonight Show Starring Jimmy Fallon,Whisper Challenge Veterans Day Reunion Surprise,2018-11-09T22:37:17Z,23,"['tonight show', 'talk', 'Jimmy Fallon', 'the ...",Jimmy Fallon partners with T-Mobile and uses a...,44324613,892947,19324,PT8M10S,4.432506e+07,4.134462e+07


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0.1         500 non-null    int64  
 1   Unnamed: 0           500 non-null    int64  
 2   channel_title        500 non-null    object 
 3   video_title          500 non-null    object 
 4   published_time       500 non-null    object 
 5   category             500 non-null    int64  
 6   tags                 484 non-null    object 
 7   description          498 non-null    object 
 8   view                 500 non-null    int64  
 9   likes                500 non-null    int64  
 10  comments             500 non-null    int64  
 11  duration             500 non-null    object 
 12  N_cal                500 non-null    float64
 13  published_time_view  500 non-null    float64
dtypes: float64(2), int64(6), object(6)
memory usage: 54.8+ KB


In [11]:
# time data revised
# input e.g. 2019-11-19T12:00:02Z

df1 = pd.DataFrame(data = df.published_time)
df1['published_time_revise'] = df1['published_time'].map(lambda x:x.split('-'))       # split published_time

for i in range(len(df1['published_time_revise'])):
    df1['published_time_revise'][i][2] = df1['published_time_revise'][i][2][:2]
    for k in range(len(df1['published_time_revise'][i])):
        df1['published_time_revise'][i][k] = int(df1['published_time_revise'][i][k])  # convert to int

# output e.g.[2019, 11, 9]

In [12]:
# counting number of days

d0 = date(df1['published_time_revise'][0][0], df1['published_time_revise'][0][1], df1['published_time_revise'][0][2])
d1 = date.today()              # get today's date
delta = d1 - d0                # get range
today_day = int(delta.days)    # convert to int

# output e.g. The number of days between the given range of dates is : 1718

In [13]:
# calculate N

N_list = []

for i in range(len(df.view)):

    revised_view = 0
    
    # revise prepared
    N = 0
    earlymkt_p = 0.2  # θ
    day_start = 0     # τ, 30 days after video uploaded

    bs = 0.99
    bw = 0.01
    βs = 4
    βw = 11
    
    part_1_noN = earlymkt_p * ((1 - e ** (-bs * today_day))/(1 + βs*e ** (-bs * today_day)))
    part_2_noN = (1 - earlymkt_p) * ((1 - e ** (-bw * (today_day))/(1 + βw*e ** (-bw * (today_day)))))
    
    N = df.view[i] / (part_1_noN + part_2_noN)
    N_list.append(N)
    
df['N_cal'] = N_list

In [14]:
# calculate view num

revised_view_list = []

for i in range(len(df.view)):
    
    revised_view = 0
    
    # revise prepared
    earlymkt_p = 0.2 # θ
    day_start = 0    # τ, 30 days after video uploaded

    bs = 0.99 # bs
    bw = 0.01 # bw
    βs = 4    # βs
    βw = 11   # βw
    
    part_1 = earlymkt_p * df['N_cal'][i] * ((1 - e ** (-bs * 7))/(1 + βs*e ** (-bs * 7)))
    part_2 = (1 - earlymkt_p) * df['N_cal'][i] * ((1 - e ** (-bw * (7 - day_start))/(1 + βw*e ** (-bw * (7 - day_start)))))

    revised_view = part_1 + part_2  # output
    revised_view_list.append(revised_view)

    
df['published_time_view'] = revised_view_list

In [10]:
outputpath = '2018to2019_comedy(23).csv'
df.to_csv(outputpath)